# Estimate the posterior of coin fairness using SVI




## Notes on Pyro Representations

When representing the different entities in the probabilistic programming like
observations, latent variables and parameters use the below functions.

1. Observations -> `pyro.sample` with `obs` argument 
2. Latent Variables -> `pyro.sample` 
3. Parameters -> `pyro.param`

**Model** is the posterior or hidden random variables $p_{\theta}(z|x)$

**Guide Function** in `pyro` is the Variation Distribution $q_{\phi}({\bf x})$

`guide` is the approximation of the posterior $p_{\theta}(z|x)$




**TEST**
1. $p_{\theta}({\bf x}, {\bf z}) = p_{\theta}({\bf x}|{\bf z}) p_{\theta}({\bf z})$

In [97]:
import math
import pyro
import pyro.distributions as dist
import torch
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO

## Coin fairness estimation using SVI

SVI - Stocastic Variational Inference


If you get a coin and want to know whether it's biased or not, how can you do that ?

1. Do some trial by doing tossing the coin and measure the observations, N number of times.
2. Pick a prior distribution, we can assume the coin is fair one. We can pick a Beta(10, 10), as it's centered around 0.5.


### Model the problem

Head and Tail are encoded as set of two valuesSet(Head(1), Tail(0))

A Random variable $X$ is encoded as fairness of the coin.  The random variable  $X$ takes real value from -> [0, 1] based on the fairness of the coin. We don't know the current distribution of Random Variable $X$, we assume ideally it has $Beta(20, 20)$ as it prior distribution. We are picking $Beta(10, 10) because its expected value comes at `0.5`.

The coin is Fair if we get equal number of head and tail on tossing.

So we have to do experiment/Trail using this coin and measure the observation and then validate the current fairness of the coin based on the observation that we got from the experiment. Use the prior distribution of the Fairness as above. 


#### Graphical Representation


In [65]:
# pyro.sample("obs", pyro.distributions.Bernoulli(0.5), obs=1.0)

In [124]:
def model(data):
    """
    Using the Likelihood and Prior estimate the Posterior.
    
    P(z|X;𝜃) = P(X|z;𝜃) * P(z;𝜃)
    """
    
    # Hyper parameters for the prior distribution, prior belief about
    # the fairness.
    alpha0 = torch.tensor(10.)
    beta0 = torch.tensor(10.)
    
    # Sample Prior beleif about the coin or P(x_fair) from Beta prior.
    x_prior = pyro.sample("latent_fairness", pyro.distributions.Beta(alpha0, beta0))
    
    # Create the Observations from the data / trails we done using the
    # given coin.
    for ind, d in enumerate(data):
        # Observed datapoint from the Bernoulli.
        # Likelihood is Bernoulli(x_prior) or P(Data|x_prior)
        pyro.sample(f"obs_{ind}", pyro.distributions.Bernoulli(x_prior), obs=torch.tensor(d))
    

def guide(data):
    """
    A Variational Distribution on Latent variable "latent_fairness".
    """
    
    # We don't know the distribution of the variational distribution, 
    # we need to find best parameters of that distribution to match the
    # posterior distribution.
    
    # Let's pick another Beta Distribution, so both prior and posterior are
    # Compatible or conjugate to each other.
    
    alpha_q = pyro.param("alpha_q", torch.tensor(15.),
                         constraint=pyro.distributions.constraints.positive)
    
    beta_q = pyro.param("beta_q", torch.tensor(15.),
                       constraint=pyro.distributions.constraints.positive)
    
    pyro.sample("latent_fairness", pyro.distributions.Beta(alpha_q, beta_q))
    
    
def sample_data():
    return [torch.tensor(1.)] * 1 + [torch.tensor(0.)] * 300


def learn_posterior(data, model, guide, n_steps=5000):
    """
    Learn the posterior from the observation. Or update our prior
    information about the latent variable is correct or not.
    """
    # Validate the params are from the right distribution.
    pyro.enable_validation(True)
    
    # Clear the param store, before starting the training.
    pyro.clear_param_store()
    
    # Optimizer settings.
    adam_param = {"lr": 0.005, "betas":(0.9, 0.999) }
    optimizer = Adam(adam_param)
    
    # Inference algorithm setup.
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
    
    
    # Do inference from observation using SVI.
    for i in range(n_steps):
        svi.step(data)
        if i % 100 == 0:
            print (".", end="")
    
    # Now we have updated parameters that we asked to learn in guide.
    # Or we found a best variational distribution by picking best
    # parameters, lets pick the parameter and get some properties of the
    # variational distribution.
    #
    alpha_q = pyro.param("alpha_q").item()
    beta_q  = pyro.param("beta_q").item()
    
    infered_mean = alpha_q / (alpha_q + beta_q)
    factor = beta_q / (alpha_q * (1. + alpha_q + beta_q))
    infered_std = infered_mean * math.sqrt(factor)
    
    print()
    print(f"Posterior parameters are -> Beta({alpha_q}, {beta_q})")
    print(f"\nBased on the Data and the Prior distribution, fairness of the " +
          f"coin is {infered_mean : 0.3f} +- {infered_std : 0.3f}"
         )

In [129]:
# sample_data()
# sum(data)

torch.nn.Module?

Init signature: torch.nn.Module()
Docstring:     
Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super(Model, self).__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            return F.relu(self.conv2(x))

Submodules assigned in this way will be registered, and will have their
parameters converted too when you call :meth:`to`, etc.
Init docstring: Initializes internal Module state, shared by both nn.Module and ScriptModule.
File:           ~/ENV3/lib/python3.6/site-packages/torch/nn/modules/module.py
Type:           type


In [125]:
data = sample_data()

learn_posterior(data, model, guide)

/home/haridas/ENV3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


..................................................
Posterior parameters are -> Beta(5.59282922744751, 152.99102783203125)

Based on the Data and the Prior distribution, fairness of the coin is  0.035 +-  0.015


In [117]:
# learn_posterior(data, model, guide)

In [118]:
# learn_posterior(data, model, guide)

In [119]:
# learn_posterior(data, model, guide)